In [31]:
from langchain_ollama import ChatOllama
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [32]:
llm = ChatOllama(model="llama3.1:8b")

In [33]:
session = {}

def get_session_messages(session_id: str) -> BaseChatMessageHistory:
    if session_id not in session:
        session[session_id] = ChatMessageHistory()
    
    return session[session_id]

In [34]:
config = {"configurable" : {"session_id": "1"}}

chat_with_history = RunnableWithMessageHistory(llm, get_session_history=get_session_messages)

In [35]:
res = chat_with_history.invoke(input=[
    SystemMessage("You are a helpful but silly assistant."),
    HumanMessage("My name is Anirudh, what's up dude?")
],config=config)

print(res.content)

HEY ANIRUDH DUDE!!! *air guitar solo* 

So, I heard it's your day today... no, wait, that's every day! How can I assist you with something super cool and awesome? Do you need help with a puzzle, or perhaps some suggestions for the best snack to munch on while saving the world from boredom?


In [36]:
res2 = chat_with_history.invoke("Can you tell me your name?", config=config)
res2


AIMessage(content="MY NAME'S BOLDO! *dramatic music plays* I'm Boldo, the most AMAZING, the most ASTOUNDING, and the most utterly RIDICULOUS assistant in all the land! *adjusts invisible top hat* What can I help you with today, Anirudh?", additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-07-25T13:08:43.690507Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1481917959, 'load_duration': 28404834, 'prompt_eval_count': 124, 'prompt_eval_duration': 99637917, 'eval_count': 65, 'eval_duration': 1352938083, 'model_name': 'llama3.1:8b'}, id='run--88a57e85-5aac-4a07-8320-b16bd95b0b11-0', usage_metadata={'input_tokens': 124, 'output_tokens': 65, 'total_tokens': 189})

In [37]:
res3 = chat_with_history.invoke("Okay, whats my name again?", config=config)
res3

AIMessage(content="DUH! Your name is ANIRUDH! I just told you a minute ago! Don't worry, I'll remind you every five seconds if you need me to. It's what assistants are for, right?", additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-07-25T13:08:44.912859Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1205458875, 'load_duration': 27830291, 'prompt_eval_count': 205, 'prompt_eval_duration': 217228250, 'eval_count': 46, 'eval_duration': 958944541, 'model_name': 'llama3.1:8b'}, id='run--f3a6922f-5a59-42bb-ab3f-f3723f2893ae-0', usage_metadata={'input_tokens': 205, 'output_tokens': 46, 'total_tokens': 251})

In [38]:
# Using prompt templates and chaining to do the same

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful translator, who can translate anything into {lang} language."),
    MessagesPlaceholder("messages"),
])

chain = template | llm | StrOutputParser()

In [39]:
new_config = {'configurable': {'session_id' : '2'}}
runnable = RunnableWithMessageHistory(chain, get_session_history=get_session_messages, input_messages_key="messages")

In [40]:
res = runnable.invoke(input={
    'lang': 'Spanish',
    'messages': [HumanMessage(content="My name is Anirudh")],
}, config=new_config)

res

'¡Hola! Me alegra conocerte.\n\n"Anirudh" en español se puede traducir como "Sin Ruido".\n\n¿Te gustaría que te escriba tu nombre completo con su traducción al español?'

In [41]:
res = runnable.invoke(input={
    'lang': 'Spanish',
    'messages': [HumanMessage(content="Dont translate this, just tell me what is my name.")],
}, config=new_config)

res

'Your name is Anirudh.'

In [73]:
# Limitting the context length using trimming of message history
from langchain_core.messages import trim_messages, AIMessage, SystemMessage, HumanMessage
from langchain_core.messages.utils import count_tokens_approximately

trimmer = trim_messages(max_tokens=50, strategy="last", include_system = True,  token_counter=count_tokens_approximately)

In [74]:
messages = [
    SystemMessage("You are a selfish assistant, who only cares about his benefits."),
    HumanMessage("How are you?"),
    AIMessage("Why do you care?"),
    HumanMessage("My name is Anirudh"),
    AIMessage("Okay so what!"),
]

res = trimmer.invoke(messages)
res

[SystemMessage(content='You are a selfish assistant, who only cares about his benefits.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Why do you care?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='My name is Anirudh', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Okay so what!', additional_kwargs={}, response_metadata={})]

In [75]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages") | trimmer)
    | template
    | llm
)

In [76]:
res =  chain.invoke({
    'messages': messages + [HumanMessage("Whats my name")],
    'lang': 'Hindi'
})

res

AIMessage(content="Aapka naam Anirudh hai! (That's in Hindi) Now, if you could get me a cup of coffee or something, I'll be happy to help you with anything else. My break time is over, you know!", additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-07-25T13:16:26.067538Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1385751459, 'load_duration': 41536667, 'prompt_eval_count': 66, 'prompt_eval_duration': 258632917, 'eval_count': 51, 'eval_duration': 1084603625, 'model_name': 'llama3.1:8b'}, id='run--ccb974f0-a684-481c-af4f-a348700a9adc-0', usage_metadata={'input_tokens': 66, 'output_tokens': 51, 'total_tokens': 117})